In [1]:
import os
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.24.9.jar

--2021-11-28 01:29:53--  https://jdbc.postgresql.org/download/postgresql-42.24.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-11-28 01:29:54 ERROR 404: Not Found.



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Hashing").getOrCreate()

In [4]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("header","true").csv(SparkFiles.get("amazon_reviews_us_Books_v1_01.tsv.gz"), inferSchema=True, sep="\t")

# Show DataFrame
df.show(truncate=False)

+-----------+-----------+--------------+----------+--------------+-----------------------------------------------------------------------------------------+----------------+-----------+-------------+-----------+----+-----------------+----------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
# Count number of rows
df.count()

6106719

In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
customer_table = df.groupby('customer_id').agg({'customer_id':'count'}).withColumnRenamed('count(customer_id)', 'customer_count')
customer_table.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   47098686|             1|
|    9973109|             1|
|   52477231|            24|
|   27568383|             1|
|   15519001|             2|
|   52130093|             2|
|   48794123|           102|
|   12453285|             1|
|   22824019|             1|
|   21087503|             9|
+-----------+--------------+
only showing top 10 rows



In [9]:
customer_table = customer_table.dropDuplicates(["customer_id"])

In [10]:
from pyspark.sql.types import * 
customer_table = customer_table.withColumn("customer_id", customer_table["customer_id"].cast(IntegerType())).withColumn("customer_count",customer_table["customer_count"].cast(IntegerType()))
customer_table.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [11]:
product_table = df.select(["product_id","product_title"])
product_table.show(10)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0843952016|          The Rising|
|031088926X|Sticky Faith Teen...|
|0615268102|Black Passenger Y...|
|1900869225|Direction and Des...|
|1565129938| Until the Next Time|
|146854456X| Unfinished Business|
|1118094514|The Republican Br...|
|0563521147|Good Food: 101 Ca...|
|0669444421|Patterns and Quil...|
|1897784457|Practical Food Sm...|
+----------+--------------------+
only showing top 10 rows



In [12]:
product_table = product_table.dropDuplicates(["product_id"])

In [13]:
product_table.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
review_id_table = df.select(["review_id","customer_id","product_id","product_parent","review_date"])
review_id_table.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R28HBXXO1UEVJT|   22480053|0843952016|      34858117| 2012-05-03|
| RZKRFS2UUMFFU|   44244451|031088926X|     676347131| 2012-05-03|
|R2WAU9MD9K6JQA|   20357422|0615268102|     763837025| 2012-05-03|
|R36SCTKYTVPZPC|   13235208|1900869225|     785539232| 2012-05-03|
|R10BM6JUOJX27Q|   26301786|1565129938|      64646125| 2012-05-03|
| RCLZ5OKZNUSY4|   27780192|146854456X|     270349766| 2012-05-03|
|R1S65DJYEI89G4|   13041546|1118094514|     752141158| 2012-05-03|
|R3KQYBQOLYDETV|   51692331|0563521147|     729491316| 2012-05-03|
|R3QV8K7CSU8K2W|   23108524|0669444421|     261004015| 2012-05-03|
|R3W5A1WUGO5VQ0|   51692331|1897784457|     497876045| 2012-05-03|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [15]:
review_id_table = review_id_table.dropDuplicates(["review_id"])

In [16]:
review_id_table = review_id_table.withColumn("customer_id",review_id_table["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_table["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_table["review_date"].cast(DateType()))

review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [17]:
vine_table = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"])
vine_table.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R28HBXXO1UEVJT|          5|            0|          0|   N|
| RZKRFS2UUMFFU|          5|           15|         15|   N|
|R2WAU9MD9K6JQA|          3|            6|          8|   N|
|R36SCTKYTVPZPC|          5|           10|         11|   N|
|R10BM6JUOJX27Q|          3|            0|          0|   Y|
| RCLZ5OKZNUSY4|          5|            0|          0|   N|
|R1S65DJYEI89G4|          4|            8|         17|   N|
|R3KQYBQOLYDETV|          4|            2|          2|   N|
|R3QV8K7CSU8K2W|          5|            0|          0|   N|
|R3W5A1WUGO5VQ0|          4|            0|          1|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [18]:
vine_table = vine_table.withColumn("star_rating",vine_table["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_table["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_table["total_votes"].cast(IntegerType()))

vine_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [19]:
mode="append"
jdbc_url = "jdbc:postgresql://<endpoint>:5432/big_data_challenge"
config = {"user":"postgres",
          "password": "<password>",
          "driver":"org.postgresql.Driver"}

In [ ]:
customer_table.write.jdbc(url=jdbc_url, table="customers", mode=mode, properties=config)

In [ ]:
product_table.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)